In [2]:
import tensorflow as tf
import os
import pandas as pd
import keras
import numpy as np
from tensorflow.keras import datasets, layers, models
from tensorflow import keras 
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,SimpleRNN
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
from keras import regularizers
from keras.layers import LSTM
from keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
import numpy as np


def eventroll_to_multihot_vector(eventroll):
    """
    Parameters
    ----------
    eventroll : np.array
        Eventroll matrix of shape=(11, 1000).
    
    Returns
    -------
    np.array
        A multihot vector of shape=(10,)
    """
    
    # findout active events:
    active_events = (eventroll.sum(axis=1) >= 0.5).astype('float')
    
    # remove silence class:
    return np.delete(active_events, 8)


# Reshaping and Loading variable X

In [1]:

dirx=os.listdir('../input/ass2new/X')
dirx.sort()
X=np.zeros([10000,64,1000])
X=X.astype('float32')
for i in range(10000):
    x=np.load('../input/ass2new/X/'+dirx[i])
    X[i]=np.reshape(x,[x.shape[1],x.shape[2]])

# Loading variable Y

In [4]:
diry=os.listdir('../input/ass2new/Y')
diry.sort()
Y=np.zeros([10000,10])
for i in range(10000):
    y=np.load('../input/ass2new/Y/'+diry[i])
    Y[i]=eventroll_to_multihot_vector(y)

In [5]:
X = X.reshape(10000,64,1000,1)

# Model making

In [12]:
# model

model = models.Sequential([
    layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(64,1000,1)),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    
    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.BatchNormalization(),
    layers.Dropout(0.25),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='sigmoid')
])

In [13]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])


# Making dictionary for changing weights in loss function

In [16]:
class_weight = {0: 6.67e-4,
                1: 0.0010070493454179,
                2: 8.86e-4,
                3: 4.09e-4,
                4: 7.08e-4,
                5: 9.12e-4,
                6: 7.41e-4,
                7: 8.20e-4,
                8: 1.08e-4,
                9: 9.56e-4}

In [17]:
model.fit( X,Y,batch_size=32,epochs=40,class_weight=class_weight)

2022-11-13 09:20:34.474993: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2560000000 exceeds 10% of free system memory.


Epoch 1/40
313/313 [==============================] - 9s 26ms/step - loss: 2.0464e-05 - precision_1: 0.9796 - recall_1: 0.9697
Epoch 2/40
313/313 [==============================] - 8s 26ms/step - loss: 2.0115e-05 - precision_1: 0.9784 - recall_1: 0.9710
Epoch 3/40
313/313 [==============================] - 8s 26ms/step - loss: 1.9801e-05 - precision_1: 0.9799 - recall_1: 0.9711
Epoch 4/40
313/313 [==============================] - 8s 26ms/step - loss: 1.9006e-05 - precision_1: 0.9794 - recall_1: 0.9711
Epoch 5/40
313/313 [==============================] - 8s 26ms/step - loss: 1.8338e-05 - precision_1: 0.9813 - recall_1: 0.9745
Epoch 6/40
313/313 [==============================] - 8s 26ms/step - loss: 1.8218e-05 - precision_1: 0.9821 - recall_1: 0.9720
Epoch 7/40
313/313 [==============================] - 8s 26ms/step - loss: 1.8116e-05 - precision_1: 0.9824 - recall_1: 0.9730
Epoch 8/40
313/313 [==============================] - 8s 26ms/step - loss: 1.7886e-05 - precision_1: 0.9812 - r

# Saving model

In [18]:
model.save('model.h5')